In [1]:
!pip install pandas

In [2]:
!pip install openpyxl

In [3]:
!pip install ipywidgets

In [4]:
!pip install IPython

In [5]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from io import BytesIO
import base64

# Carga el archivo
uploader = widgets.FileUpload(accept='.xlsx, .xltx', multiple=False)
display(widgets.HTML("<h3>📋 Sube tu archivo Excel para organizar puestos y sugerir unificaciones:</h3>"))
display(uploader)

output = widgets.Output()
display(output)

def encontrar_fila_encabezados(df_raw):
    for i, row in df_raw.iterrows():
        encabezados = [str(c).strip().upper() for c in row.values]
        if "CENTRO DE TRABAJO" in encabezados and "DEPARTAMENTO" in encabezados and "PUESTO DE TRABAJO" in encabezados:
            return i
    return None

def crear_boton_descarga(nombre_archivo):
    with open(nombre_archivo, "rb") as f:
        data = f.read()
    b64 = base64.b64encode(data).decode()
    enlace_html = f"""
    <a download="{nombre_archivo}" href="data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{b64}" target="_blank">
        📥 Descargar archivo generado
    </a>
    """
    return widgets.HTML(enlace_html)

def procesar_archivo(change):
    output.clear_output()
    with output:
        if uploader.value:
            try:
                uploaded_file = next(iter(uploader.value.values())) if isinstance(uploader.value, dict) else uploader.value[0]
                content = BytesIO(uploaded_file['content'])
                df_raw = pd.read_excel(content, sheet_name=0, header=None)

                fila_encabezados = encontrar_fila_encabezados(df_raw)
                if fila_encabezados is None:
                    print("❌ No se encontraron las columnas necesarias.")
                    return

                content.seek(0)
                df = pd.read_excel(content, sheet_name=0, header=fila_encabezados)

                # Detectar nombres de columnas
                def match_column(nombre, columnas):
                    for c in columnas:
                        if nombre in str(c).strip().upper():
                            return c
                    return None

                col_centro = match_column("CENTRO DE TRABAJO", df.columns)
                col_dep = match_column("DEPARTAMENTO", df.columns)
                col_puesto = match_column("PUESTO DE TRABAJO", df.columns)

                if not col_centro or not col_dep or not col_puesto:
                    print("❌ No se pudieron localizar todas las columnas clave.")
                    return

                df_filtrado = df[[col_centro, col_dep, col_puesto]].dropna()

                # Agrupar y contar personas por puesto
                df_agrupado = (
                    df_filtrado
                    .groupby([col_centro, col_dep, col_puesto])
                    .size()
                    .reset_index(name="NÚMERO DE PERSONAS")
                    .sort_values(by=[col_centro, col_dep, col_puesto])
                )

                # Añadir advertencias y sugerencias
                advertencias = []
                sugerencias = []

                for i, row in df_agrupado.iterrows():
                    centro = row[col_centro]
                    depto = row[col_dep]
                    puesto = row[col_puesto]
                    cantidad = row["NÚMERO DE PERSONAS"]

                    if cantidad > 2:
                        advertencias.append("")
                        sugerencias.append("")
                        continue

                    advertencias.append("⚠️ Bajo número de personas")

                    # Buscar otro puesto similar en el mismo departamento
                    similares = df_agrupado[
                        (df_agrupado[col_centro] == centro) &
                        (df_agrupado[col_dep] == depto) &
                        (df_agrupado[col_puesto] != puesto) &
                        (df_agrupado["NÚMERO DE PERSONAS"] > 3)
                    ]

                    if not similares.empty:
                        sugerencias.append(f"Unificar con: {similares.iloc[0][col_puesto]}")
                    else:
                        similares_centro = df_agrupado[
                            (df_agrupado[col_centro] == centro) &
                            (df_agrupado[col_puesto] != puesto) &
                            (df_agrupado["NÚMERO DE PERSONAS"] > 3)
                        ]
                        if not similares_centro.empty:
                            sugerencias.append(f"Unificar con: {similares_centro.iloc[0][col_puesto]}")
                        else:
                            sugerencias.append("Unificar con otro puesto del centro")

                # Agregar columnas al dataframe
                df_agrupado["ADVERTENCIA"] = advertencias
                df_agrupado["PROPUESTA DE UNIFICACIÓN"] = sugerencias

                # Fila total
                total_personas = df_agrupado["NÚMERO DE PERSONAS"].sum()
                fila_total = pd.DataFrame({
                    col_centro: ["TOTAL"],
                    col_dep: [""],
                    col_puesto: [""],
                    "NÚMERO DE PERSONAS": [total_personas],
                    "ADVERTENCIA": [""],
                    "PROPUESTA DE UNIFICACIÓN": [""]
                })

                df_final = pd.concat([df_agrupado, fila_total], ignore_index=True)

                print("✅ Archivo procesado correctamente:")
                display(df_final)

                # Guardar a Excel
                nombre_archivo = "puestos_ordenados_por_centro.xlsx"
                with pd.ExcelWriter(nombre_archivo, engine="openpyxl") as writer:
                    df.to_excel(writer, sheet_name="Original", index=False)
                    df_final.to_excel(writer, sheet_name="Resumen Puestos", index=False)

                # Mostrar botón de descarga
                display(crear_boton_descarga(nombre_archivo))

            except Exception as e:
                print("❌ Error al procesar el archivo:", e)

# Asociar evento al uploader
uploader.observe(procesar_archivo, names='value')

HTML(value='<h3>📋 Sube tu archivo Excel para organizar puestos y sugerir unificaciones:</h3>')

FileUpload(value=(), accept='.xlsx, .xltx', description='Upload')

Output()